## Set Up Environment

In [ ]:
# Cell 0

network = "custom"  # set this to one of mainnet, yeouido, euljiro, pagoda, or custom
contracts_file_custom = 'contracts_20210124194550.pkl'

connections = {
"mainnet": {"iconservice": "https://ctz.solidwallet.io",       "nid": 1},
"yeouido": {"iconservice": "https://bicon.net.solidwallet.io", "nid": 3},
"euljiro": {"iconservice": "https://test-ctz.solidwallet.io",  "nid": 2},
"pagoda":  {"iconservice": "https://zicon.net.solidwallet.io", "nid":80},
"custom":  {"iconservice": "http://52.53.175.151:9000",       "nid": 3}}

env = connections[network]

In [ ]:
# Cell 1

from iconsdk.exception import JSONRPCException
from iconsdk.libs.in_memory_zip import gen_deploy_data_content
from iconsdk.icon_service import IconService
from iconsdk.providers.http_provider import HTTPProvider
from iconsdk.builder.transaction_builder import CallTransactionBuilder, TransactionBuilder, DeployTransactionBuilder
from iconsdk.builder.call_builder import CallBuilder
from iconsdk.signed_transaction import SignedTransaction
from iconsdk.wallet.wallet import KeyWallet
from iconsdk.utils.convert_type import convert_hex_str_to_int
from repeater import retry
from shutil import make_archive
import pickle as pkl
from datetime import datetime
import json
import os

ICX = 1000000000000000000 # 10**18
GOVERNANCE_ADDRESS = "cx0000000000000000000000000000000000000000"
ORACLE = "cx61a36e5d10412e03c907a507d1e8c6c3856d9964"

@retry(JSONRPCException, tries=10, delay=1, back_off=2)
def get_tx_result(_tx_hash):
    tx_result = icon_service.get_transaction_result(_tx_hash)
    return tx_result

In [ ]:
str(HTTPProvider(env["iconservice"], 3))

In [ ]:
# Cell 2

icon_service = IconService(HTTPProvider(env["iconservice"], 3))
NID = env["nid"]

In [ ]:
# Cell 3

wallet = KeyWallet.load("keystores/keystore_test1.json", "test1_Account")
# Balanced test wallet
with open("keystores/balanced_test.pwd", "r") as f:
    key_data = f.read()
btest_wallet = KeyWallet.load("keystores/balanced_test.json", key_data)

### Send ICX to Balanced test wallet

In [ ]:
# Cell 4
# Only necessary if running locally or on the private tbears server.

test_wallet = KeyWallet.load("./keystores/keystore_test1.json", "test1_Account")
transaction = TransactionBuilder()\
    .from_(wallet.get_address())\
    .to(btest_wallet.get_address())\
    .value(1000000 * ICX)\
    .step_limit(1000000) \
    .nid(NID) \
    .nonce(2) \
    .version(3) \
    .build()
signed_transaction = SignedTransaction(transaction, test_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

In [ ]:
wallet = btest_wallet

### Deployed to testnet

In [ ]:
# Cell 5
# The following addresses are those deployed to the testnet

# Uncomment these lines and comment out the definition below to load contracts from a saved file.
# with open(contracts_file_custom, 'rb') as f:
#     contracts = pkl.load(f)

contracts = {'loans': {'zip': 'core_contracts/loans.zip',
  'SCORE': 'cxcd9ae763fabcf6c3c9a475f727264e9cfffdfaf2'},
 'staking': {'zip': 'core_contracts/staking.zip',
  'SCORE': 'cx52aab6b1dd316396e684573464af36c634059721'},
 'dividends': {'zip': 'core_contracts/dividends.zip',
  'SCORE': 'cx02bba98bf78c91329c6958c61ceed1e7bad3b68a'},
 'reserve_fund': {'zip': 'core_contracts/reserve_fund.zip',
  'SCORE': 'cx418f4a6392b88568cb9587c6ca638b358f8ca728'},
 'rewards': {'zip': 'core_contracts/rewards.zip',
  'SCORE': 'cx65fb9d6c9bafdc9cdbe6bf258fd31b90f2f4ab8d'},
 'dex': {'zip': 'core_contracts/dex.zip',
  'SCORE': 'cx291dacbb875a94b364194a5febaac4e6318681f7'},
 'governance': {'zip': 'core_contracts/governance.zip',
  'SCORE': 'cx4179247372b8ce4c9edd8013dd3918852bd0c7fa'},
 'dummy_oracle': {'zip': 'core_contracts/dummy_oracle.zip',
  'SCORE': 'cx61a36e5d10412e03c907a507d1e8c6c3856d9964'},
 'sicx': {'zip': 'token_contracts/sicx.zip',
  'SCORE': 'cx5b0d6a5b244e9254ca05592389672c5185d9f5f8'},
 'icd': {'zip': 'token_contracts/icd.zip',
  'SCORE': 'cx699694fa8dbb76d904d33b63f90dee7cf0440e71'},
 'bal': {'zip': 'token_contracts/bal.zip',
  'SCORE': 'cxd89295b894b72b3b5966848f6fe2bbb997f99e71'},
 'bwt': {'zip': 'token_contracts/bwt.zip',
  'SCORE': 'cx6fda38cfc5c828763306ad7fb4225a8062bd0aee'}}

In [ ]:
# Private tbears server. Saved in custom_contracts_20210201083826.pkl

contracts = {'loans': {'zip': 'core_contracts/loans.zip',
  'SCORE': 'cxf57c89fec090ece16863208f9aad39d0af45f292'},
 'staking': {'zip': 'core_contracts/staking.zip',
  'SCORE': 'cx6ebb33adc84ffc01324a981d4069e7fe03553836'},
 'dividends': {'zip': 'core_contracts/dividends.zip',
  'SCORE': 'cxe678a2da0c4db8b68797e1390a26f26ab7bca5bb'},
 'reserve_fund': {'zip': 'core_contracts/reserve_fund.zip',
  'SCORE': 'cxf38b453627eb74f95e9c2bc006933a34734c2354'},
 'rewards': {'zip': 'core_contracts/rewards.zip',
  'SCORE': 'cx2b3df4e5016a70c91119376cd331938cb5274645'},
 'dex': {'zip': 'core_contracts/dex.zip',
  'SCORE': 'cx73f31d5b38051176ec4f230d9898ab106879638c'},
 'governance': {'zip': 'core_contracts/governance.zip',
  'SCORE': 'cxa4aaf2749b813899bf9866e8051f7577ee944f8a'},
 'dummy_oracle': {'zip': 'core_contracts/dummy_oracle.zip',
  'SCORE': 'cx56b6ab9ac9898b3008816a0ec9c36b4a5bde515d'},
 'sicx': {'zip': 'token_contracts/sicx.zip',
  'SCORE': 'cx9c6325d2a9e7d52cfa1bd5ae3d970c338dcc2086'},
 'icd': {'zip': 'token_contracts/icd.zip',
  'SCORE': 'cx84ce744362692b29f6985e0878cfe2b711707ec7'},
 'bal': {'zip': 'token_contracts/bal.zip',
  'SCORE': 'cxdf23b3e7aefdf2111119239959677e95b4d443db'},
 'bwt': {'zip': 'token_contracts/bwt.zip',
  'SCORE': 'cx6fb254adae3621b52fce8722b6ffe4ad8314ac1e'}}

In [ ]:
# Cell 6

deploy = list(contracts.keys())[:]
deploy.remove('dummy_oracle')
for directory in {"core_contracts", "token_contracts"}:
    with os.scandir(directory) as it:
        for file in it:
            archive_name = directory + "/" + file.name
            if file.is_dir() and file.name in deploy:
                make_archive(archive_name, "zip", directory, file.name)
                contracts[file.name]['zip'] = archive_name + '.zip'

contracts

## Deploy All SCOREs

In [ ]:
# Cell 7
# prints out contracts dictionary with all new contract addresses

step_limit = 4000100000

for score in deploy:
    loc = contracts[score]
    deploy_transaction = DeployTransactionBuilder()\
        .from_(wallet.get_address())\
        .to(GOVERNANCE_ADDRESS)\
        .nid(NID)\
        .nonce(100)\
        .content_type("application/zip")\
        .content(gen_deploy_data_content(loc["zip"]))\
        .build()

#     step_limit = icon_service.estimate_step(deploy_transaction) + 100000
    
    signed_transaction = SignedTransaction(deploy_transaction, wallet, step_limit)
    tx_hash = icon_service.send_transaction(signed_transaction)

    res = get_tx_result(tx_hash)
    contracts[score]["SCORE"] = res.get('scoreAddress', '')
    print(score)
    print(f'Status: {res["status"]}')
    if len(res["eventLogs"]) > 0:
        for item in res["eventLogs"]:
            print(f'{item} \n')
    if res['status'] == 0:
        print(f'Failure: {res["failure"]}')

contracts

In [ ]:
# Cell 8
# saves contracts dictionary to file

# now = datetime.utcnow()
# filename = f'contracts_{now.year}{str(now.month).zfill(2)}{str(now.day).zfill(2)}{str(now.hour).zfill(2)}{str(now.minute).zfill(2)}{str(now.second).zfill(2)}.pkl'
# print(f'saving contracts list as {filename}')
# with open(filename, 'wb') as f:
#     pkl.dump(contracts, f)

In [ ]:
# Cell 9
# Prints out dictionary of SCORE addresses for use in testing UI

def get_scores_json(contracts):
    scores = {}
    for score in contracts:
        scores[score] = contracts[score]['SCORE']
    return json.dumps(scores)

print(get_scores_json(contracts))

### Deploy a single SCORE

In [ ]:
# Cell 10

# Deploy a single SCORE
contract = 'staking'
update = contracts[contract]['SCORE']
zip_file = contracts[contract]['zip']
deploy_transaction = DeployTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(GOVERNANCE_ADDRESS)\
    .nid(NID)\
    .nonce(100)\
    .content_type("application/zip")\
    .content(gen_deploy_data_content(zip_file))\
    .build()

step_limit = 10000000000


signed_transaction = SignedTransaction(deploy_transaction, wallet, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

## Update a SCORE

In [ ]:
# Cell 11

contract = 'loans'
update = contracts[contract]['SCORE']
zip_file = contracts[contract]['zip']
deploy_transaction = DeployTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(update)\
    .nid(NID)\
    .nonce(100)\
    .content_type("application/zip")\
    .content(gen_deploy_data_content(zip_file))\
    .build()

# step_limit = icon_service.estimate_step(deploy_transaction) + 100000
step_limit = 3000000000

signed_transaction = SignedTransaction(deploy_transaction, wallet, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

## Set parameters

In [ ]:
# Cell 12

from typing import List, TypedDict

addresses = {'loans': contracts['loans']['SCORE'],
             'dex': contracts['dex']['SCORE'],
             'staking': contracts['staking']['SCORE'],
             'rewards': contracts['rewards']['SCORE'],
             'reserve_fund': contracts['reserve_fund']['SCORE'],
             'dividends': contracts['dividends']['SCORE'],
             'oracle': contracts['dummy_oracle']['SCORE'],
             'sicx': contracts['sicx']['SCORE'],
             'icd': contracts['icd']['SCORE'],
             'baln': contracts['bal']['SCORE'],
             'bwt': contracts['bwt']['SCORE']
            }

class DistPercentDict(TypedDict):
    recipient_name : str
    bal_token_dist_percent: int

_recipient_list: List[DistPercentDict] = [{'recipient_name': 'Loans', 'bal_token_dist_percent': 25 * 10**16},
                   {'recipient_name': 'SICXICX', 'bal_token_dist_percent': 10 * 10**16},
                   {'recipient_name': 'SICXICD', 'bal_token_dist_percent': 20 * 10**16},
                   {'recipient_name': 'BALNICD', 'bal_token_dist_percent': 20 * 10**16},
                   {'recipient_name': 'Worker Tokens', 'bal_token_dist_percent': 20 * 10**16},
                   {'recipient_name': 'Reserve Fund', 'bal_token_dist_percent': 5 * 10**16}
                  ]

settings = [{'contract': 'sicx', 'method': 'setAdmin', 'params':{'_admin': contracts['staking']['SCORE']}},
            {'contract': 'sicx', 'method': 'setStakingAddress', 'params':{'_address': contracts['staking']['SCORE']}},
            {'contract': 'icd', 'method': 'setAdmin', 'params':{'_admin': contracts['loans']['SCORE']}},
            {'contract': 'icd', 'method': 'setOracle', 'params':{'_address': contracts['dummy_oracle']['SCORE'], '_name': 'BandChain'}},
            {'contract': 'bal', 'method': 'setAdmin', 'params':{'_admin': wallet.get_address()}},
            {'contract': 'bal', 'method': 'mint', 'params':{'_amount': 10 * ICX}},
            {'contract': 'bwt', 'method': 'setAdmin', 'params':{'_admin': contracts['governance']['SCORE']}},
            {'contract': 'bwt', 'method': 'setBalnToken', 'params':{'_address': contracts['bal']['SCORE']}},
            {'contract': 'staking', 'method': 'setSicxAddress', 'params':{'_address': contracts['sicx']['SCORE']}},
            {'contract': 'reserve_fund', 'method': 'setLoansScore', 'params':{'_address': contracts['loans']['SCORE']}},
            {'contract': 'reserve_fund', 'method': 'setBalnToken', 'params':{'_address': contracts['bal']['SCORE']}},
            {'contract': 'reserve_fund', 'method': 'setSicxToken', 'params':{'_address': contracts['sicx']['SCORE']}},
            {'contract': 'bal', 'method': 'mintTo', 'params':{'_account': contracts['reserve_fund']['SCORE'], '_amount': 1000 * ICX}},
            {'contract': 'bal', 'method': 'mintTo', 'params':{'_account': wallet.get_address(), '_amount': 1000 * ICX}},
            {'contract': 'bal', 'method': 'setAdmin', 'params':{'_admin': contracts['rewards']['SCORE']}},
            {'contract': 'dividends', 'method': 'setLoansScore', 'params':{'_address': contracts['loans']['SCORE']}},
            {'contract': 'rewards', 'method': 'setBalnAddress', 'params':{'_address': contracts['bal']['SCORE']}},
            {'contract': 'rewards', 'method': 'setBwtAddress', 'params':{'_address': contracts['bwt']['SCORE']}},
            {'contract': 'rewards', 'method': 'setReserveAddress', 'params':{'_address': contracts['reserve_fund']['SCORE']}},
            {'contract': 'rewards', 'method': 'setBatchSize', 'params':{'_batch_size': 30}},
            {'contract': 'rewards', 'method': 'setGovernance', 'params':{'_address': contracts['governance']['SCORE']}},
            {'contract': 'rewards', 'method': 'addNewDataSource', 'params':{'_data_source_name': 'Loans', '_contract_address': contracts['loans']['SCORE']}},
            {'contract': 'rewards', 'method': 'addNewDataSource', 'params':{'_data_source_name': 'SICXICX', '_contract_address': contracts['dex']['SCORE']}},
            {'contract': 'rewards', 'method': 'addNewDataSource', 'params':{'_data_source_name': 'SICXICD', '_contract_address': contracts['dex']['SCORE']}},
            {'contract': 'rewards', 'method': 'addNewDataSource', 'params':{'_data_source_name': 'BALNICD', '_contract_address': contracts['dex']['SCORE']}},
            {'contract': 'rewards', 'method': 'updateBalTokenDistPercentage', 'params':{'_recipient_list': _recipient_list}},
            {'contract': 'dex', 'method': 'setGovernance', 'params':{'_address': contracts['governance']['SCORE']}},
            {'contract': 'dex', 'method': 'setAdmin', 'params':{'_admin': wallet.get_address()}},
            {'contract': 'dex', 'method': 'setRewards', 'params':{'_address': contracts['rewards']['SCORE']}},
            {'contract': 'dex', 'method': 'setStaking', 'params':{'_address': contracts['staking']['SCORE']}},
            {'contract': 'dex', 'method': 'setSicx', 'params':{'_address': contracts['sicx']['SCORE']}},
            {'contract': 'dex', 'method': 'setDividends', 'params':{'_address': contracts['dividends']['SCORE']}},
            {'contract': 'dex', 'method': 'setIcd', 'params':{'_address': contracts['icd']['SCORE']}},
            {'contract': 'loans', 'method': 'setAdmin', 'params':{'_admin': wallet.get_address()}},
            {'contract': 'loans', 'method': 'setGovernance', 'params':{'_address': contracts['governance']['SCORE']}},
            {'contract': 'loans', 'method': 'setDividends', 'params':{'_address': contracts['dividends']['SCORE']}},
            {'contract': 'loans', 'method': 'setReserve', 'params':{'_address': contracts['reserve_fund']['SCORE']}},
            {'contract': 'loans', 'method': 'setRewards', 'params':{'_address': contracts['rewards']['SCORE']}},
            {'contract': 'loans', 'method': 'setStaking', 'params':{'_address': contracts['staking']['SCORE']}},
            {'contract': 'loans', 'method': 'addAsset', 'params':{'_token_address': contracts['sicx']['SCORE'], 'is_active': 1, 'is_collateral': 1}},
            {'contract': 'loans', 'method': 'addAsset', 'params':{'_token_address': contracts['icd']['SCORE'], 'is_active': 1, 'is_collateral': 0}},
            {'contract': 'loans', 'method': 'addAsset', 'params':{'_token_address': contracts['bal']['SCORE'], 'is_active': 0, 'is_collateral': 1}},
            {'contract': 'governance', 'method': 'setAddresses', 'params':{'_addresses': addresses}},
            {'contract': 'governance', 'method': 'launchBalanced', 'params':{}}]

In [ ]:
# Cell 13

results = {}
for sett in settings:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(wallet.get_address())\
        .to(contracts[sett['contract']]['SCORE'])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)

    res = get_tx_result(tx_hash)
    results[f'{sett["contract"]}|{sett["method"]}|{sett["params"]}'] = res
    print(f'Status: {res["status"]}')
    if len(res["eventLogs"]) > 0:
        for item in res["eventLogs"]:
            print(f'{item} \n')
    if res['status'] == 0:
        print(f'Failure: {res["failure"]}')

## Run through read-only methods on all SCOREs

In [ ]:
# Cell 14

ro_methods = {"loans": ["name", "getAssetTokens", "getCollateralTokens", "getTotalCollateral", "getAvailableAssets", "assetCount"],
              "staking": ["name", "getSicxAddress", "getTodayRate"],
              "dividends": ["name", "getBalances"],
              "reserve_fund": ["name", "getBalances"],
              "rewards": ["name", "getDataSourceNames"],
              "governance": ["name", "getAddresses"],
              "sicx": ["name", "symbol", "decimals", "totalSupply", "getAdmin", "lastPriceInLoop"],
              "icd": ["name", "symbol", "decimals", "totalSupply", "getAdmin"],
              "bal": ["name", "symbol", "decimals", "totalSupply", "getAdmin"]
              }

In [ ]:
# Cell 15
# Runs through the readonly methods defined above. These can also be viewed using the testing UI

results = {}
for score in ro_methods.keys():
    print()
    print(f'SCORE: {score}')
    print(f'Address: {contracts[score]["SCORE"]}')
    results[score] = {}
    for method in ro_methods[score]:
        call = CallBuilder().from_(wallet.get_address())\
                            .to(contracts[score]['SCORE'])\
                            .method(method)\
                            .build()
        result = icon_service.call(call)
        if method in ['totalSupply', 'decimals', 'get_total_collateral', 'get_rate']:
            result = int(result, 0)
            if method in ['totalSupply', 'get_total_collateral', 'get_rate']:
                result = result / ICX
        print(f'{method}: {result}')
        results[score][method] = result

### Get sICX from Staking

In [ ]:
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['staking']['SCORE'])\
    .value(800 * ICX)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("addCollateral")\
    .params({})\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

## Deposit ICX and Mint ICD Loan

In [ ]:
# Cell 16

data1 = "{\"method\": \"_deposit_and_borrow\", \"params\": {\"_sender\": \"".encode("utf-8")
data2 = "\", \"_asset\": \"ICD\", \"_amount\": 40000000000000000000}}".encode("utf-8")
params = {'_data1': data1, '_data2': data2}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['loans']['SCORE'])\
    .value(800 * ICX)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("addCollateral")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

### Just deposit collateral

In [ ]:
# Cell 17

data1 = "{\"method\": \"_deposit_and_borrow\", \"params\": {\"_sender\": \"".encode("utf-8")
data2 = "\", \"_asset\": \"\", \"_amount\": 0}}".encode("utf-8")
params = {'_data1': data1, '_data2': data2}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['loans']['SCORE'])\
    .value(30 * ICX)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("addCollateral")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

### Repay Some ICD

In [ ]:
# Cell 18

data = "{\"method\": \"_repay_loan\", \"params\": {}}".encode("utf-8")
params = {'_to': contracts['loans']['SCORE'], '_value': 5 * ICX, '_data': data}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['icd']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

### Withdraw Some Collateral

In [ ]:
# Cell 19

params = {'_value': 5 * ICX}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['loans']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("withdrawCollateral")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if 'eventLogs' in res.keys():
    print(f'EventLogs: {res["eventLogs"]}')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

### Test Liquidation

In [ ]:
# Cell 20
# 1. Deposit collateral to fresh wallet

data1 = "{\"method\": \"_deposit_and_borrow\", \"params\": {\"_sender\": \"".encode("utf-8")
data2 = "\", \"_asset\": \"\", \"_amount\": 0}}".encode("utf-8")
params = {'_data1': data1, '_data2': data2}
transaction = CallTransactionBuilder()\
    .from_(btest_wallet.get_address())\
    .to(contracts['loans']['SCORE'])\
    .value(782769 * ICX // 1000)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("addCollateral")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, btest_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if 'eventLogs' in res.keys():
    print(f'EventLogs: {res["eventLogs"]}')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
# Cell 21
# 2. Check the account standing after depositing collateral - Should be Undetermined, and
#    holding '0x2a6f1a22364bbe8000' worth of sICX.

params = {'_owner': btest_wallet.get_address()}
call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['loans']['SCORE'])\
                    .method('getAccountPositions')\
                    .params(params)\
                    .build()
result = icon_service.call(call)
result

#### 3. Update the loans SCORE to add a loan of ICD to the Balanced test wallet
Code in the on_update() method will mint and add the ICD. Adds an undercollateralized amount of ICD, 200 ICD.

In [ ]:
# Cell 22
# 3a. May need to run this a few times to get the position below liquidate threshold.

contract = 'loans'
update = contracts[contract]['SCORE']
zip_file = contracts[contract]['zip']
deploy_transaction = DeployTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(update)\
    .nid(NID)\
    .nonce(100)\
    .content_type("application/zip")\
    .content(gen_deploy_data_content(zip_file))\
    .build()

# step_limit = icon_service.estimate_step(deploy_transaction) + 100000
step_limit = 3000000000

signed_transaction = SignedTransaction(deploy_transaction, wallet, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if 'eventLogs' in res.keys():
    print(f'EventLogs: {res["eventLogs"]}')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
# Cell 23
# 4. Check the new standing of the account. Should have added '0xad78ebc5ac6200000' ICD and have standing of 'Liquidate'.

params = {'_owner': btest_wallet.get_address()}
call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['loans']['SCORE'])\
                    .method('getAccountPositions')\
                    .params(params)\
                    .build()
result = icon_service.call(call)
result

In [ ]:
# Cell 24
# 5. Liquidate the account position.

params = {'_owner': btest_wallet.get_address()}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['loans']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("liquidate")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if 'eventLogs' in res.keys():
    print(f'EventLogs: {res["eventLogs"]}')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
# Cell 25
# 6. Check the standing of the account after liquidation. Should now have zero balance for sICX and ICD.

params = {'_owner': btest_wallet.get_address()}
call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['loans']['SCORE'])\
                    .method('getAccountPositions')\
                    .params(params)\
                    .build()
result = icon_service.call(call)
result

### DEX SICX Test

Does the following:

1. Sends ICX to the DEX
2. Checks overall ICX balance
3. Checks withdrawal lock
4. Mints some SICX
5. Trades SICX against ICX

In [ ]:
# Cell 26

transaction = TransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['dex']['SCORE'])\
    .value(1 * ICX)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(101)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash
print(get_tx_result(tx_hash))


In [ ]:
# Cell 27

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getICXBalance')\
                    .params({'_address': wallet.get_address()}) \
                    .build()
result = icon_service.call(call)
int(result, 0) / 10**18

In [ ]:
# Cell 28

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getICXWithdrawLock')\
                    .build()
result = icon_service.call(call)
int(result, 0) / 10**18

In [ ]:
# Cell 29

params = {'_to': wallet.get_address()}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['staking']['SCORE'])\
    .value(10 * ICX)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("addCollateral")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash
print(get_tx_result(tx_hash))


In [ ]:
# Cell 30

data = "{\"method\": \"_swap_icx\"}".encode("utf-8")
params = {'_to': contracts['dex']['SCORE'], '_value': 2 * ICX, '_data': data}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['sicx']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash
print(get_tx_result(tx_hash))


In [ ]:
# Cell 31

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getICXBalance')\
                    .params({'_address': wallet.get_address()}) \
                    .build()
result = icon_service.call(call)
int(result, 0) / 10**18

In [ ]:
# Cell 32

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getNamedPools')\
                    .build()
result = icon_service.call(call)
print(result)

In [ ]:
# Cell 33

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getDataBatch')\
                    .params({'_name': 'SICXICX', '_limit': 10, '_snapshot_id': 1}) \
                    .build()
result = icon_service.call(call)
print(result)

In [ ]:
# Cell 34

data = "{\"method\": \"_deposit\"}".encode("utf-8")
params = {'_to': contracts['dex']['SCORE'], '_value': 2 * ICX, '_data': data}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['sicx']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash
print(get_tx_result(tx_hash))


In [ ]:
# Cell 35

data = "{\"method\": \"_deposit\"}".encode("utf-8")
params = {'_to': contracts['dex']['SCORE'], '_value': 2 * ICX, '_data': data}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['icd']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash
print(get_tx_result(tx_hash))


In [ ]:
# Cell 36

data = "{\"method\": \"_deposit\"}".encode("utf-8")
params = {'_to': contracts['dex']['SCORE'], '_value': 100 * ICX, '_data': data}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['bal']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash
print(get_tx_result(tx_hash))


In [ ]:
# Cell 37

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getDeposit')\
                    .params({'_user': wallet.get_address(), '_tokenAddress': contracts['icd']['SCORE']}) \
                    .build()
result = icon_service.call(call)
int(result, 0) / 10**18

In [ ]:
# Cell 38

transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['dex']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("add")\
    .params({'_baseToken': contracts['sicx']['SCORE'], '_quoteToken': contracts['icd']['SCORE'], '_baseValue': 1 * ICX, '_quoteValue': 1 * ICX}) \
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash
print(get_tx_result(tx_hash))


In [ ]:
# Cell 39

transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['dex']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("permit")\
    .params({'_pid': 1, '_permission': 1}) \
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash
print(get_tx_result(tx_hash))


In [ ]:
# Cell 40

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getInversePrice')\
                    .params({'_pid': 2}) \
                    .build()
result = icon_service.call(call)
int(result, 0) / 10**18

In [ ]:
# Cell 41

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getPoolTotal')\
                    .params({'_pid': 2, '_token': contracts['icd']['SCORE']}) \
                    .build()
result = icon_service.call(call)
int(result, 0) / 10**18

In [ ]:
# Cell 42

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getPoolId')\
                    .params({'_token1Address': contracts['sicx']['SCORE'], '_token2Address': contracts['icd']['SCORE'] }) \
                    .build()
result = icon_service.call(call)
print("pool w/ tokens: " + str(contracts['sicx']['SCORE']) + ' & ' + str(contracts['icd']['SCORE']))
print(result)

In [ ]:
# Cell 43

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getPoolBase')\
                    .params({'_pid': 2}) \
                    .build()
result = icon_service.call(call)
print(result)

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getPoolQuote')\
                    .params({'_pid': 2}) \
                    .build()
result = icon_service.call(call)
print(result)

In [ ]:
# Cell 44

transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['dex']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("setMarketName")\
    .params({'_pid':2, '_name': 'SICXICD'}) \
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash
print(get_tx_result(tx_hash))

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getNamedPools')\
                    .build()
result = icon_service.call(call)
print(result)

In [ ]:
# Cell 45

to_token = contracts['sicx']['SCORE']
params_data = "{\"method\": \"_swap\", \"params\": {\"toToken\":\"" + str(to_token) + "\", \"maxSlippage\":10}}"
data = params_data.encode("utf-8")
params = {'_to': contracts['dex']['SCORE'], '_value': (int) (1 * ICX / 100), '_data': data}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['icd']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash
print(get_tx_result(tx_hash))


In [ ]:
# Cell 45

to_token = contracts['icd']['SCORE']
params_data = "{\"method\": \"_swap\", \"params\": {\"toToken\":\"" + str(to_token) + "\", \"maxSlippage\":250}}"
data = params_data.encode("utf-8")
params = {'_to': contracts['dex']['SCORE'], '_value': (int) (1 * ICX / 100), '_data': data}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['sicx']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash
print(get_tx_result(tx_hash))


In [ ]:
# Cell 46

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getPoolTotal')\
                    .params({'_pid': 2, '_token': contracts['sicx']['SCORE']}) \
                    .build()
result = icon_service.call(call)
int(result, 0) / 10**18

In [ ]:
# Cell 47

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getPoolTotal')\
                    .params({'_pid': 2, '_token': contracts['icd']['SCORE']}) \
                    .build()
result = icon_service.call(call)
int(result, 0) / 10**18

In [ ]:
# Cell 48

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('totalSupply')\
                    .params({'_pid': 2}) \
                    .build()
result = icon_service.call(call)
int(result, 0) / 10**18

In [ ]:
# Cell 49

transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['dex']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("remove")\
    .params({'_pid':2, '_value': 1 * ICX, '_withdraw': 1}) \
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash
print(get_tx_result(tx_hash))

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getPoolTotal')\
                    .params({'_pid': 2, '_token': contracts['icd']['SCORE']}) \
                    .build()
result = icon_service.call(call)
int(result, 0) / 10**18

In [ ]:
# Cell 50

data = "{\"method\": \"_deposit\"}".encode("utf-8")
params = {'_to': contracts['dex']['SCORE'], '_value': 30 * ICX, '_data': data}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['icd']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash
print(get_tx_result(tx_hash))


In [ ]:
# Cell 51

transaction = TransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['dex']['SCORE'])\
    .value(1 * ICX)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(101)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash
print(get_tx_result(tx_hash))


In [ ]:
# Cell 52

transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['dex']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("cancelSicxicxOrder")\
    .params({}) \
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash
print(get_tx_result(tx_hash))


In [ ]:
# Cell 38

transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['dex']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("add")\
    .params({'_baseToken': contracts['bal']['SCORE'], '_quoteToken': contracts['icd']['SCORE'], '_baseValue': 100 * ICX, '_quoteValue': 10 * ICX}) \
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash
print(get_tx_result(tx_hash))


In [ ]:
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['dex']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("setMarketName")\
    .params({'_pid':3, '_name': 'BALNICD'}) \
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash
print(get_tx_result(tx_hash))

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getNamedPools')\
                    .build()
result = icon_service.call(call)
print(result)

In [ ]:
to_token = contracts['bal']['SCORE']
params_data = "{\"method\": \"_swap\", \"params\": {\"toToken\":\"" + str(to_token) + "\", \"maxSlippage\":250}}"
data = params_data.encode("utf-8")
params = {'_to': contracts['dex']['SCORE'], '_value': (int) (1 * ICX), '_data': data}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['icd']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash
print(get_tx_result(tx_hash))
